In [1]:
#%pip install shap

In [2]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
class XAIModels:
  model = None
  score = None
  train_data=None
  test_data=None

  #xai를 구현 시 필요한 파라미터를 적용
  def __init__(self, model):
    self.model=model
  def novelty_contribution(self, train_data, test_data, score, threshold):
    self.train_data = train_data
    self.test_data = test_data
    self.score = score
    self.threshold = threshold

    model_df = self.test_data.copy()
    model_df['anomaly_score']=self.score
    model_df.sort_values(by='anomaly_score', axis=0, ascending=False, inplace=True)
    novelties_df=model_df[model_df['anomaly_score']>threshold]
    novelties_df_index= novelties_df.index
    shap_list = []
    for idx in novelties_df_index:
      record_to_explain = self.test_data.iloc[idx]
      explainer = shap.KernelExplainer(self.model.decision_function, self.train_data.head(100).values)
      shap_values = explainer.shap_values(record_to_explain, nsamples='auto')
      shap_list.append(shap_values)
    shap_values_all = pd.DataFrame(data = shap_list)
    shap_values_all.index=novelties_df_index.to_list()
    shap_values_all.columns= self.test_data.columns
    shap.initjs()
    shap.summary_plot(shap_values_all.values, shap_values_all.columns, plot_type = "bar")